In [ ]:
from glass.ete.otol.pg import osm_to_lulc
from glass.prop.prj import get_epsg
import datetime
import os

osm = '/mnt/d/osm2lulc02/osmdata/osm_terceira.xml'

nomenclature = 'clc'

refraster = '/mnt/d/osm2lulc02/lmt/rst_terceira.tif'

result = '/mnt/d/osm2lulc02/lulc_fmosm/terceira.gpkg'

db = '/mnt/d/osm2lulc02/lulc_fmosm/terceira_26062023.sql'

ws = '/mnt/d/osm2lulc02/debug'

time_a = datetime.datetime.now().replace(microsecond=0)

res, logs = osm_to_lulc(
    osm, nomenclature, get_epsg(refraster),
    result, savedb=db, tmpfld=ws
)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
#shp = '/mnt/d/osm2lulc02/lulc_fmosm/terceira.gpkg'
shp = '/mnt/disk1/jasp/terceira.gpkg'

lyr = 'osmtolulc_v2'

col = 'lulc'

osm_db = 'osm_terceira'

In [ ]:
import datetime
import copy

from glass.ete.otol.tools import classes_priority
from glass.sql.q import q_to_obj
from glass.prop.sql import row_num
from glass.gp.ovl.sql import st_erase
from glass.it.db import gpkg_lyr_attr_to_psql

In [ ]:
order_cls = classes_priority(2)

table_cls = gpkg_lyr_attr_to_psql(shp, lyr, col, osm_db, 'tblcls')

In [ ]:
refname = copy.deepcopy(table_cls)

for e in range(len(order_cls)):
    if e + 1 == len(order_cls): break

    if order_cls[e] not in table_cls: continue

    for i in range(e+1, len(order_cls)):
        if order_cls[i] not in table_cls: continue

        time_a = datetime.datetime.now().replace(microsecond=0)

        table_cls[order_cls[i]] = st_erase(
            osm_db,
            table_cls[order_cls[i]],
            table_cls[order_cls[e]],
            "geom", "geom",
            otbl=f"{refname[order_cls[i]]}_{str(e)}",
            method=1
        )

        time_b = datetime.datetime.now().replace(microsecond=0)

        print(f'{table_cls[order_cls[i]]} <-> {table_cls[order_cls[e]]} || {time_b - time_a}')
        print('---------------------------------')

        nrows = row_num(osm_db, table_cls[order_cls[i]], api='psql')

        # Create Geometry index for the new table

        if not nrows:
            del table_cls[order_cls[i]]

In [ ]:
order_cls

In [ ]:
table_cls

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ntbl = st_erase(osm_db, "tblcls_28", "tblcls_86", "geom", "geom", otbl="metodo_1")

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ntbl = st_erase(osm_db, "tblcls_28", "tblcls_86", "geom", "geom", otbl="metodo_2", method=2)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ntbl = st_erase(osm_db, "tblcls_28", "tblcls_86", "geom", "geom", otbl="metodo_3", method=3)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
from glass.ete.otol.tools import module_osmtags

tags = module_osmtags(2)

In [ ]:
tags[tags.name == 'roads'].lulc_id.unique()

In [ ]:
tags.name.unique()

In [ ]:
tags[
    (tags['value'] == 'yes') &
    (tags['key'] == 'building')
]

In [ ]:
from glass.ete.osm2lulc.grs import osm_to_lulc
from glass.ete.osm2lulc.utils import record_time_consumed
import datetime
import os
from glass.pys.oss import fprop, mkdir
from glass.pys.tm import now_as_str

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ws = mkdir(os.path.join(
    os.path.dirname(osm),
    fprop(result, 'fn')
), overwrite=True)

shp, time_check = osm_to_lulc(
    osm, nomenclature, refraster,
    result, tmpfld=ws,
    overwrite=True, savedb=db
)

time_b = datetime.datetime.now().replace(microsecond=0)
    
# Record time consumed in xlsx table
record_time_consumed(time_check, os.path.join(
    os.path.dirname(result),
    fprop(result, 'fn') + '.xlsx'
))

print(time_b - time_a)